In [1]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
from tensorflow.keras.utils import plot_model

In [2]:
# data loading config
batch_size = 64
img_height = 100
img_width = 100
labels = 'inferred'
label_mode = 'categorical'  # sparse one hot encoding
color_mode = 'rgb'
shuffle = True
seed = 69
val_split = 0.2
AUTOTUNE = tf.data.AUTOTUNE

print('Training data:')
train = tf.keras.utils.image_dataset_from_directory('train', labels=labels, label_mode=label_mode,
color_mode=color_mode, shuffle=shuffle, seed=seed, image_size=(img_height, img_width), batch_size=batch_size, validation_split=val_split, subset='training')
train_class_names = train.class_names

print('\nValidation data:')
val = tf.keras.utils.image_dataset_from_directory('train', labels=labels, label_mode=label_mode,
color_mode=color_mode, shuffle=shuffle, seed=seed, image_size=(img_height, img_width), batch_size=batch_size, validation_split=val_split, subset='validation')
val_class_names = val.class_names

train = train.cache().prefetch(buffer_size=AUTOTUNE)
val = val.cache().prefetch(buffer_size=AUTOTUNE)

Training data:
Found 3081 files belonging to 34 classes.
Using 2465 files for training.

Validation data:
Found 3081 files belonging to 34 classes.
Using 616 files for validation.


In [3]:
# model layers
xIn = Input((img_height, img_width, 3))
effnet = tf.keras.applications.efficientnet_v2.EfficientNetV2M(include_top=False, weights='imagenet', input_shape=(img_height, img_width, 3),)
x = effnet(xIn)
x = Flatten()(x)
x = Dropout(0.3)(x)
x = Dense(256, activation='swish')(x)
x = Dropout(0.3)(x)
x = Dense(256, activation='swish')(x)
x = Dropout(0.3)(x)
xOut = Dense(len(train_class_names))(x)


model = Model(inputs=xIn, outputs=xOut)

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-3),
    loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True, label_smoothing=0.1),
    metrics=['accuracy'])

callbacks = [
    tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=20, restore_best_weights=True),
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy', factor=0.1, patience=15, verbose=1)
]

model.summary()
model.fit(train, epochs=100, batch_size=64, validation_data=val, callbacks=callbacks)

214212608/214201816 [==============================] - 10s 0us/step
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 100, 100, 3)]     0         
                                                                 
 efficientnetv2-m (Functiona  (None, 4, 4, 1280)       53150388  
 l)                                                              
                                                                 
 flatten (Flatten)           (None, 20480)             0         
                                                                 
 dropout (Dropout)           (None, 20480)             0         
                                                                 
 dense (Dense)               (None, 256)               5243136   
                                                                 
 dropout_1 (Dropout)         (None, 256)               0   

In [4]:
model.save('comnist')

INFO:tensorflow:Assets written to: comnist\assets


In [23]:
test = tf.keras.utils.image_dataset_from_directory('test', labels=None, label_mode=None,
color_mode=color_mode, shuffle=False, seed=seed, image_size=(img_height, img_width), batch_size=batch_size)
filenames = test.file_paths
filenames = [file.split('\\')[-1].split('.jpg')[0] for file in filenames]
test = test.cache().prefetch(buffer_size=AUTOTUNE)
preds = model.predict(test)
preds = tf.nn.softmax(preds)
preds = tf.argmax(preds, axis=-1)

Found 12399 files belonging to 1 classes.


In [28]:
preds_dict = dict()
for i in range(len(preds)):
    preds_dict.update({filenames[i]: train_class_names[int(preds[i])]})

import pandas as pd
pred_df = pd.read_csv('test.csv')
pred_df_unordered = pd.DataFrame.from_dict(preds_dict, columns=['label'], orient='index')
pred_df_unordered.reset_index(inplace=True)
pred_df_unordered.columns = ['id', 'label']

pred_df_aligned = pred_df.merge(pred_df_unordered, on='id', how='left')
pred_df_aligned.drop(['label_x'], axis=1, inplace=True)
pred_df_aligned.columns = ['id', 'label']
pred_df_aligned.to_csv('submission.csv', index=False)

In [29]:
# Connect to graders
import sys
# insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(1, 'C:/Users/alien/Documents/PyCharm Projects/Cyberthon 2021/pyctfsglib.py')
import pyctfsglib as ctfsg
import random

USER_TOKEN = "WrlLCkymxwtgFwRHZsdmKfSwcdqIpnqoXEtRkciVRZJfBJUgcEJoxVZjNTQRdqkR" # You need to fill this up
GRADER_URL = random.choice([
  "http://chals.cyberthon22t.ctf.sg:50201/",
  "http://chals.cyberthon22t.ctf.sg:50202/"
])

grader = ctfsg.DSGraderClient(GRADER_URL, USER_TOKEN)
grader.submitFile('submission.csv')

DSGraderClient: Successfully Connected!
[SERVER] MOTD: CHECK your USER_TOKEN and GRADER_URL HTTP address! I'm CoMNIST @4d907f5f7099
ProofOfWork Challenge =>  ('CTFSGRB6c90f7d7df10511e295d8b4964b873a1', 22)
ProofOfWork Answer Found! =>  271565


'{"challenge":{"name":"CoMNIST"},"id":"cl22nyk6fchue0832ftq93rod","status":"PARTIALLY_CORRECT","multiplier":0.9665,"submittedBy":{"username":"hci-69"},"createdAt":"2022-04-17T02:23:06Z"}'